# Preperation

In [1]:
import torch
import matplotlib.pyplot as plt
import yaml
import os
import sys
from copy import deepcopy
import h5py
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import numpy as np
import argparse


from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve
from glob import glob

In [2]:
cd ../../..

/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN


In [3]:
cd src

/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/src


In [4]:
from model import CINN
from trainer import ECAETrainer
import data_util
from documenter import Documenter
from plotter import Plotter
from matplotlib import cm
from myDataLoader import MyDataLoader
from calc_obs import *
from model import CINN
import plotting
import HighLevelFeatures as HLF
from XMLHandler import XMLHandler

In [5]:
import os
import argparse

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

import pandas as pd
from matplotlib import cm
# from matplotlib.transforms import Bbox

import data_util
from calc_obs import *
import math
import torch

from evaluate_plotting_helper import *

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.default'] = 'rm'
plt.rcParams['text.usetex'] = True

labelfont = FontProperties()
labelfont.set_family('serif')
labelfont.set_name('Times New Roman')
labelfont.set_size(20)

axislabelfont = FontProperties()
axislabelfont.set_family('serif')
axislabelfont.set_name('Times New Roman')
axislabelfont.set_size(20)

tickfont = FontProperties()
tickfont.set_family('serif')
tickfont.set_name('Times New Roman')
tickfont.set_size(20)


In [6]:
cd ..

/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN


In [7]:
def plot_hist(
        file_name,
        data,
        reference,
        p_ref='photon',
        axis_label=None,
        xscale='linear',
        yscale='log',
        vmin=None,
        vmax=None,
        n_bins=50,
        ymin=None,
        ymax=None,
        ax=None,
        panel_ax=None,
        panel_scale="linear",
        density=True,
        labels=None,
        errorbars_true=False,
        errorbars_fake=False):

    if type(errorbars_fake) == bool:
        errorbars_fake = [errorbars_fake]
    
    if type(data)==list and type(data[0])==np.ndarray:
        data_list = data
    else:
        data_list = [data]
        
    if len(errorbars_fake) != len(data_list):
        assert len(errorbars_fake) == 1, "Wrong size for the errorbars index"
        
        errorbars_fake = [errorbars_fake[0] for _ in range(len(data_list))]
    
    for i in range(len(data_list)):
        data_list[i] = data_list[i][np.isfinite(data_list[i])]
    
    reference = reference[np.isfinite(reference)]

    all_data = data_list + [reference]

    # Set the plotting boundaries
    if vmin is None:
        vmin = np.inf
        for elem in all_data:
            vmin = np.min([np.min(elem), vmin])
    if vmax is None:
        vmax = -np.inf
        for elem in all_data:
            vmax = np.max([np.max(elem), vmax])
            
    # Get the bins (Modifications needed if logscale is used)
    if xscale=='log':
        
        if vmin==0:
            vmin = np.inf     
            for elem in all_data:
                vmin = np.min([np.min(elem[elem>1e-7]), vmin])
                
        if isinstance(n_bins, int):
            bins = np.logspace(np.log10(vmin), np.log10(vmax), n_bins)
        else:
            bins = n_bins
    else:
        if isinstance(n_bins, int):
            bins = np.linspace(vmin, vmax, n_bins)
        else:
            bins = n_bins
    
    colors = cm.gnuplot2(np.linspace(0.2, 0.8, 3))
    if p_ref == 'electron':
        color = colors[0]
    elif p_ref == 'photon':
        color = colors[1]
    elif p_ref == 'pion':
        color = colors[2]
    else:
        color = 'blue'
        
    create_fig = False
    if ax is None:
        create_fig = True
        fig, ax = plt.subplots(1,1,figsize=(6,6))    
        
        
    # Plot the reference data
    if not errorbars_true:
        ns_1, bins_1, patches_1 = ax.hist(reference, bins=bins, histtype='stepfilled',
                alpha=0.5, color=color, density=density, label='GEANT')
    else:
        dup_last = lambda a: np.append(a, a[-1])

        bins_1 = bins
        
        counts, _ = np.histogram(reference, bins_1, density=False)
        ns_1, _ = np.histogram(reference, bins_1, density=True)
        
        mask = (counts == 0)
        counts[mask] = 1
        
        ref_err = ns_1 / np.sqrt(counts)
            
        ref_err[mask] = 0
        
        ax.step(bins_1, dup_last(ns_1), color="blue", alpha=1,
                        linewidth=1, where='post', label='GEANT')
        
        ax.step(bins_1, dup_last(ns_1 - ref_err), color="blue", alpha=0.5,
                        linewidth=0.5, where='post')
        ax.step(bins_1, dup_last(ns_1 + ref_err), color="blue", alpha=0.5,
                        linewidth=0.5, where='post')

        ax.fill_between(bins_1, dup_last(ns_1 - ref_err), dup_last(ns_1 + ref_err), 
                        facecolor="blue", alpha=0.3, step='post')
    
    # Plot the generated data
    alt_colors = ["green", "red", "pink"]
    
    for i, data in enumerate(data_list):
        if not errorbars_fake[i]:
            
            # Modify the labels
            if labels is None:
                label = "VAE"
            else:
                label = labels[i]
            
            # Add the first trainer to the panel and use the default color code
            if i == 0:
                ns_0, bins_0, patches_0 = ax.hist(data, bins=bins, histtype='step', linewidth=2,
                    alpha=1, density=density, label=label, color=color)
            if i == 1:
                ns_2, bins_2, patches_2 = ax.hist(data, bins=bins, histtype='step', linewidth=2,
                    alpha=1, density=density, label=label, color=color)
            else:
                ax.hist(data, bins=bins, histtype='step', linewidth=2,
                    alpha=1, density=density, label=label, color=alt_colors[i])
    
        else:
            # labels
            if labels is None:
                label = "VAE"
            else:
                label = labels[i]
                
            data = data_list[i]
            
            dup_last = lambda a: np.append(a, a[-1])
            
            if i == 0:
                bins_0 = bins
                
                counts, _ = np.histogram(data, bins_0, density=False)
                ns_0, _ = np.histogram(data, bins_0, density=True)
                
                mask = (counts == 0)
                counts[mask] = 1
                
                data_err = ns_0 / np.sqrt(counts)
                    
                data_err[mask] = 0
                
                ax.step(bins_0, dup_last(ns_0), color=alt_colors[i], alpha=1,
                                linewidth=1, where='post', label=label)
                
                ax.step(bins_0, dup_last(ns_0 - data_err), color=alt_colors[i], alpha=0.5,
                                linewidth=0.5, where='post')
                ax.step(bins_0, dup_last(ns_0 + data_err), color=alt_colors[i], alpha=0.5,
                                linewidth=0.5, where='post')

                ax.fill_between(bins_0, dup_last(ns_0 - data_err), dup_last(ns_0 + data_err), 
                                facecolor=alt_colors[i], alpha=0.3, step='post')
            elif i == 1:
                bins_2 = bins
                
                counts, _ = np.histogram(data, bins_2, density=False)
                ns_2, _ = np.histogram(data, bins_2, density=True)
                
                mask = (counts == 0)
                counts[mask] = 1
                
                data_err = ns_2 / np.sqrt(counts)
                    
                data_err[mask] = 0
                
                ax.step(bins_2, dup_last(ns_2), color=alt_colors[i], alpha=1,
                                linewidth=1, where='post', label=label)
                
                ax.step(bins_2, dup_last(ns_2 - data_err), color=alt_colors[i], alpha=0.5,
                                linewidth=0.5, where='post')
                ax.step(bins_2, dup_last(ns_2 + data_err), color=alt_colors[i], alpha=0.5,
                                linewidth=0.5, where='post')

                ax.fill_between(bins_0, dup_last(ns_2 - data_err), dup_last(ns_2 + data_err), 
                                facecolor=alt_colors[i], alpha=0.3, step='post')
            
            else:                
                counts, _ = np.histogram(data, bins, density=False)
                ns, _ = np.histogram(data, bins, density=True)
                
                mask = (counts == 0)
                counts[mask] = 1
                
                data_err = ns / np.sqrt(counts)
                    
                data_err[mask] = 0
                
                ax.step(bins, dup_last(ns), color=alt_colors[i], alpha=1,
                                linewidth=1, where='post', label=label)
                
                ax.step(bins, dup_last(ns - data_err), color=alt_colors[i], alpha=0.5,
                                linewidth=0.5, where='post')
                ax.step(bins, dup_last(ns + data_err), color=alt_colors[i], alpha=0.5,
                                linewidth=0.5, where='post')

                ax.fill_between(bins_0, dup_last(ns - data_err), dup_last(ns + data_err), 
                                facecolor=alt_colors[i], alpha=0.3, step='post')
                
        

    if panel_ax is not None:
        assert len(bins_0) == len(bins_1)
        assert (bins_0 - bins_1 < 1.e-7).all()
        
        # prevent divisions by 0! Set these bars to 0
        mask = ns_1 == 0
        ns_1[mask] = 1
        panel_data = ns_0/ns_1
        
        panel_data[mask] = 0
        
        widths = 1.2*(bins_1[1:] - bins_1[:-1])
        panel_ax.axhline(1, color="red", ls="--")
        panel_ax.hist(bins_0[:-1], bins[1:]-widths, weights=panel_data, histtype="step", lw=2, label='Generated/GEANT')
        

        panel_data = ns_2/ns_1
        
        panel_data[mask] = 0
        
        widths = 1.2*(bins_1[1:] - bins_1[:-1])
        panel_ax.axhline(1, color="red", ls="--")
        panel_ax.hist(bins_0[:-1], bins[1:]-widths, weights=panel_data, histtype="step", lw=2, label='Reconstructed/GEANT')
        
    ax.set_yscale(yscale)
    ax.set_xscale(xscale)
    if panel_ax is not None:
        panel_ax.set_yscale(panel_scale)
        panel_ax.set_xscale(xscale)

    ax.set_xlim([vmin,vmax])
    if panel_ax is not None:
        panel_ax.set_xlim([vmin,vmax])
        panel_ax.set_ylim([0.5, 1.5])
        
    if ymin is not None or ymax is not None:
        ax.set_ylim((ymin, ymax))

    if panel_ax is not None:
        panel_ax.legend()

    if axis_label:
        if panel_ax is None:
            ax.set_xlabel(axis_label, fontproperties=axislabelfont)
        else:
            panel_ax.set_xlabel(axis_label, fontproperties=axislabelfont)
            
    plt.xticks(fontproperties=tickfont)
    plt.yticks(fontproperties=tickfont)
    
    ax.legend()

    if create_fig:
        fig.tight_layout()
        fig.savefig(file_name, bbox_inches='tight')
        
    # Why this line?
    if panel_ax is None:
        plt.close()

def plot_all_hist(x_true, c_true, x_fakes, c_fakes, params, layer_boundaries, plot_dir, plot_name,
                  single_plots=False, summary_plot=True, labels=None, errorbars_true=False, errorbars_fake=False):
    
    
    threshold = params.get("threshold", 1.e-4)
    
    # Load the hlf classes
    hlf_true = data_util.get_hlf(x_true, c_true, params["particle_type"], layer_boundaries, threshold=threshold, dataset=params.get("dataset", 1))
    
    hlf_fakes = []
    
    for x_fake, c_fake in zip(x_fakes, c_fakes):
        hlf_fakes.append(data_util.get_hlf(x_fake, c_fake, params["particle_type"], layer_boundaries, threshold=threshold, dataset=params.get("dataset", 1)))
    
    os.makedirs(plot_dir, exist_ok=True)
    
    plots = plotting.get_all_plot_parameters(hlf_true, params)
    
    for plot in plots:
        plot[3]["vmin"] = None
        plot[3]["vmax"] = None

    
    # Plot all the histogramms in one file
    number_of_plots = len(plots)
    rows = number_of_plots // 6
    if number_of_plots%6 != 0:
        rows += 1
    heights = [1, 0.3, 0.3]*rows

    fig, axs = plt.subplots(rows*3,6, dpi=500, figsize=(6*7,6*np.sum(heights)), gridspec_kw={'height_ratios': heights})

    iteration = 0
    for i in range(rows*3):
        
        if i%3 == 1:
            iteration -= 6
            
        for j in range(6):
            
            if i % 3 == 2:
                # Add one (small) invisible plot as whitespace
                axs[i,j].set_visible(False)
                continue
            
            elif iteration >= number_of_plots:
                    # Plots are empty remove them
                    axs[i,j].set_visible(False)
                    iteration += 1
                    continue
            
            
            # Select the correct plot input for this axis
            function, name, args1, args2 = plots[iteration]
            
            
            if i % 3 == 0:
                # plot the main data
                plot_hist(
                        file_name=None,
                        data=[function(hlf_fake, **args1) for hlf_fake in hlf_fakes],
                        reference=function(hlf_true, **args1),
                        ax=axs[i,j],
                        panel_ax=axs[i+1,j],
                        labels=labels,
                        errorbars_fake=errorbars_fake,
                        errorbars_true=errorbars_true,
                        **args2)
                
                # Hide the (shared) x-axis
                axs[i,j].xaxis.set_visible(False)
                
                # Hide the first tick label
                plt.setp(axs[i,j].get_yticklabels()[0], visible=False)  
                iteration += 1

            if i % 3 == 1:
                plt.setp(axs[i,j].get_yticklabels()[-1], visible=False)
                iteration += 1             

    fig.subplots_adjust(hspace=0)
    # fig.savefig(os.path.join(os.path.join(plot_dir,"../"), "final.pdf"), bbox_inches='tight', dpi=500)
    fig.savefig(plot_dir+plot_name, bbox_inches='tight', dpi=500)
    # Dont use tight_layout!
    plt.close()
 

In [8]:
def load_trainer(directory, use_cuda=True, threshold=1.e-10):
    use_cuda = torch.cuda.is_available() and use_cuda
    device = 'cuda:0' if use_cuda else 'cpu' 


    with open(directory + "/params.yaml") as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
        
    doc = Documenter(params['run_name'], existing_run=True, basedir=directory,
                    log_name="log_jupyter.txt", read_only=True)
    
    particle = params.get("particle_type", "piplus")

    dataset = params.get("dataset", 1)
    if "data_path" not in params:
        particle_type = params.get("particle_type", "phoron")
        if dataset == 1:
            # Load 2nd dataset to make sure we use no training showers
            data_path = f"/remote/gpu06/ernst/Master_Thesis/Datasets/Dataset1/dataset_1_{particle_type}s_2.hdf5"
        else:
            data_path = f"/remote/gpu06/ernst/Master_Thesis/Datasets/Dataset{dataset}/dataset_{dataset}_1.hdf5"
        params["data_path"] = data_path
    
    params["threshold"] = threshold
    params["vae_dir"] = os.path.join(directory, "VAE")
    
    trainer = ECAETrainer(params, device, doc, vae_dir=os.path.join(directory, "VAE"))
    
    return trainer, params, device, doc


# Start calculations

In [9]:
!cat $PBS_GPUFILE

gpu05-gpu3


In [10]:
!nvidia-smi

Fri May 26 13:35:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 525.85.05    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 73%   87C    P2   228W / 250W |   6150MiB / 11264MiB |     85%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:03:00.0 Off |                  N/A |
| 62%   

In [12]:
glob("/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/*")

['/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_10_gamma',
 '/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_4_gamma',
 '/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_5_gamma',
 '/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_7_gamma',
 '/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_6_gamma',
 '/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_9_gamma',
 '/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_

In [13]:
directories = glob("/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/*")

for directory in directories:
    trainer, params, device, doc = load_trainer(directory)
    trainer.load()
    
    x = torch.cat((trainer.vae_trainer.train_loader.data, trainer.vae_trainer.test_loader.data), axis=0)
    c = torch.cat((trainer.vae_trainer.train_loader.cond, trainer.vae_trainer.test_loader.cond), axis=0)
    
    x_gen, c_gen = trainer.generate(len(x))
    x_reco = trainer.vae_trainer.get_reco(x, c)

    base_path = "/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/notebooks/plots/09___1.6e-01_hyperparam_tests"
    
    e_incs = torch.unique(c[..., 0])
    i = params["e_inc_index"]
    print(i, e_incs[0])
    plot_dir = base_path + "/"
    
    gamma = np.log10(params["VAE_gamma"])
    plot_name = f"gamma_{gamma:0.1e}.pdf"
    
    plot_all_hist(x, c, [x_gen, x_reco], [c_gen, c], params, layer_boundaries=trainer.layer_boundaries,
                  labels=["Generated", "Reconstructed"], errorbars_true=True, errorbars_fake=[True, False],
                  plot_dir=plot_dir, plot_name=plot_name, single_plots=False, summary_plot=True)

Using the directory: /remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_10_gamma
Using the directory: /remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_10_gamma/VAE
cuda:0
1
Use incident energy 0.32768 (10000 events)
(120991, 368) (120991, 11)
(10000, 368) (10000, 11)
CVAE(
  (noise_layer_in): noise_layer()
  (encoder): Sequential(
    (fc0): Linear(in_features=374, out_features=700, bias=True)
    (relu0): ReLU()
    (dropout0): Dropout(p=0.0, inplace=False)
    (fc1): Linear(in_features=700, out_features=400, bias=True)
    (relu1): ReLU()
    (dropout1): Dropout(p=0.0, inplace=False)
    (fc2): Linear(in_features=400, out_features=150, bias=True)
    (relu2): ReLU()
    (dropout2): Dropout(p=0.0, inplace=False)
    (fc_mu_logvar): Linear(in_features=150, out_features=100, bias=True)
  )
  (decoder): Sequential(
 

/tmp/ipykernel_1545288/4061494415.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if type(data)==list and type(data[0]==np.ndarray):


Using the directory: /remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_4_gamma
Using the directory: /remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_4_gamma/VAE
cuda:0
1
Use incident energy 0.32768 (10000 events)
(120991, 368) (120991, 11)
(10000, 368) (10000, 11)
CVAE(
  (noise_layer_in): noise_layer()
  (encoder): Sequential(
    (fc0): Linear(in_features=374, out_features=700, bias=True)
    (relu0): ReLU()
    (dropout0): Dropout(p=0.0, inplace=False)
    (fc1): Linear(in_features=700, out_features=400, bias=True)
    (relu1): ReLU()
    (dropout1): Dropout(p=0.0, inplace=False)
    (fc2): Linear(in_features=400, out_features=150, bias=True)
    (relu2): ReLU()
    (dropout2): Dropout(p=0.0, inplace=False)
    (fc_mu_logvar): Linear(in_features=150, out_features=100, bias=True)
  )
  (decoder): Sequential(
   

/tmp/ipykernel_1545288/4061494415.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if type(data)==list and type(data[0]==np.ndarray):


Using the directory: /remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_5_gamma
Using the directory: /remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_05_26_ECAE_seperated_by_einc_different_gamma/Use_einc_number_7_5_gamma/VAE
cuda:0
1
Use incident energy 0.32768 (10000 events)
(120991, 368) (120991, 11)
(10000, 368) (10000, 11)
CVAE(
  (noise_layer_in): noise_layer()
  (encoder): Sequential(
    (fc0): Linear(in_features=374, out_features=700, bias=True)
    (relu0): ReLU()
    (dropout0): Dropout(p=0.0, inplace=False)
    (fc1): Linear(in_features=700, out_features=400, bias=True)
    (relu1): ReLU()
    (dropout1): Dropout(p=0.0, inplace=False)
    (fc2): Linear(in_features=400, out_features=150, bias=True)
    (relu2): ReLU()
    (dropout2): Dropout(p=0.0, inplace=False)
    (fc_mu_logvar): Linear(in_features=150, out_features=100, bias=True)
  )
  (decoder): Sequential(
   